In [11]:
import xml.etree.ElementTree as ET

# 네임스페이스 등록
namespaces = {'PDBx': 'http://pdbml.pdb.org/schema/pdbx-v50.xsd'}

# XML 파일 열기
tree = ET.parse("C:/Users/kbjoo/Downloads/4hhb.xml")
root = tree.getroot()

# 예: atom_site 항목 하나씩 순회
for atom_site in root.findall('.//PDBx:atom_site', namespaces):
    atom_id = atom_site.attrib.get('id')
    atom_name = atom_site.find('PDBx:auth_atom_id', namespaces).text
    res_name = atom_site.find('PDBx:auth_comp_id', namespaces).text
    x = atom_site.find('PDBx:Cartn_x', namespaces).text
    y = atom_site.find('PDBx:Cartn_y', namespaces).text
    z = atom_site.find('PDBx:Cartn_z', namespaces).text
    print(f'ID: {atom_id}, Atom: {atom_name}, Residue: {res_name}, Coords: ({x}, {y}, {z})')


ID: 1, Atom: N, Residue: VAL, Coords: (19.323, 29.727, 42.781)
ID: 2, Atom: CA, Residue: VAL, Coords: (20.141, 30.469, 42.414)
ID: 3, Atom: C, Residue: VAL, Coords: (21.664, 29.857, 42.548)
ID: 4, Atom: O, Residue: VAL, Coords: (21.985, 29.541, 43.704)
ID: 5, Atom: CB, Residue: VAL, Coords: (19.887, 31.918, 43.524)
ID: 6, Atom: CG1, Residue: VAL, Coords: (20.656, 32.850, 42.999)
ID: 7, Atom: CG2, Residue: VAL, Coords: (18.692, 31.583, 43.506)
ID: 8, Atom: N, Residue: LEU, Coords: (22.284, 30.338, 41.520)
ID: 9, Atom: CA, Residue: LEU, Coords: (23.773, 29.985, 41.873)
ID: 10, Atom: C, Residue: LEU, Coords: (24.698, 31.236, 42.725)
ID: 11, Atom: O, Residue: LEU, Coords: (24.359, 32.232, 42.013)
ID: 12, Atom: CB, Residue: LEU, Coords: (24.180, 29.902, 40.374)
ID: 13, Atom: CG, Residue: LEU, Coords: (24.177, 28.761, 39.582)
ID: 14, Atom: CD1, Residue: LEU, Coords: (24.140, 27.432, 40.098)
ID: 15, Atom: CD2, Residue: LEU, Coords: (24.210, 28.919, 37.923)
ID: 16, Atom: N, Residue: SER, Coord

In [13]:
import xml.etree.ElementTree as ET

# 네임스페이스 등록
ns = {'PDBx': 'http://pdbml.pdb.org/schema/pdbx-v50.xsd'}

# XML 파일 읽기
tree = ET.parse('C:/Users/kbjoo/Downloads/4hhb-noatom.xml')
root = tree.getroot()

# chem_comp (화학 성분 정보) 추출
print("=== Chemical Components ===")
for chem in root.findall('.//PDBx:chem_comp', ns):
    comp_id = chem.attrib.get('id')
    name = chem.find('PDBx:name', ns).text
    formula = chem.find('PDBx:formula', ns).text
    weight = chem.find('PDBx:formula_weight', ns).text
    print(f"{comp_id}: {name}, Formula: {formula}, Weight: {weight}")


=== Chemical Components ===
ALA: ALANINE, Formula: C3 H7 N O2, Weight: 89.093
ARG: ARGININE, Formula: C6 H15 N4 O2 1, Weight: 175.209
ASN: ASPARAGINE, Formula: C4 H8 N2 O3, Weight: 132.118
ASP: ASPARTIC ACID, Formula: C4 H7 N O4, Weight: 133.103
CYS: CYSTEINE, Formula: C3 H7 N O2 S, Weight: 121.158
GLN: GLUTAMINE, Formula: C5 H10 N2 O3, Weight: 146.144
GLU: GLUTAMIC ACID, Formula: C5 H9 N O4, Weight: 147.129
GLY: GLYCINE, Formula: C2 H5 N O2, Weight: 75.067
HEM: PROTOPORPHYRIN IX CONTAINING FE, Formula: C34 H32 Fe N4 O4, Weight: 616.487
HIS: HISTIDINE, Formula: C6 H10 N3 O2 1, Weight: 156.162
HOH: WATER, Formula: H2 O, Weight: 18.015
LEU: LEUCINE, Formula: C6 H13 N O2, Weight: 131.173
LYS: LYSINE, Formula: C6 H15 N2 O2 1, Weight: 147.195
MET: METHIONINE, Formula: C5 H11 N O2 S, Weight: 149.211
PHE: PHENYLALANINE, Formula: C9 H11 N O2, Weight: 165.189
PO4: PHOSPHATE ION, Formula: O4 P -3, Weight: 94.971
PRO: PROLINE, Formula: C5 H9 N O2, Weight: 115.130
SER: SERINE, Formula: C3 H7 N O3,